In [1]:
from model import Model
from util import convert_tokens, get_batch_dataset, get_dataset, get_record_parser
import tensorflow as tf
from config import flags
import numpy as np
import json

In [2]:
flags.DEFINE_string('f', 'give up already', 'who cares lol')
config = flags.FLAGS

In [3]:
test_eval_file = 'data/retraining_dev_meta.json'
test_record_file = 'data/retraining_dev.tf'

with open(test_eval_file, "r") as fh:
    eval_file = json.load(fh)

meta = {'total': 1000}

In [4]:
with open(config.word_emb_file, "r") as fh:
    word_mat = np.array(json.load(fh), dtype=np.float32)
with open(config.char_emb_file, "r") as fh:
    char_mat = np.array(json.load(fh), dtype=np.float32)

In [5]:
parser = get_record_parser(config)
train_dataset = get_batch_dataset('data/retraining_train.tf', parser, config)
dev_dataset = get_dataset('data/retraining_dev.tf', parser, config)
handle = tf.placeholder(tf.string, shape=[])
iterator = tf.data.Iterator.from_string_handle(
    handle, train_dataset.output_types, train_dataset.output_shapes)
train_iterator = train_dataset.make_one_shot_iterator()
dev_iterator = dev_dataset.make_one_shot_iterator()

In [6]:
model = Model(config, iterator, word_mat, char_mat)

Instructions for updating:
Use the retry module or similar alternatives.


In [7]:
sess_config = tf.ConfigProto(allow_soft_placement=True)
sess_config.gpu_options.allow_growth = True

sess = tf.Session(config=sess_config)

In [8]:
sess.run(tf.global_variables_initializer())
saver = tf.train.Saver(var_list=[v for v in tf.global_variables() if not v.name.startswith('encoding_1/') and not v.name.startswith('badptr') and '/Adam' not in v.name and 'beta' not in v.name])
print('Restoring', tf.train.latest_checkpoint(config.save_dir))
saver.restore(sess, tf.train.latest_checkpoint(config.save_dir))
sess.run(tf.assign(model.is_train, tf.constant(True, dtype=tf.bool)))

Restoring log/model/model_60000.ckpt
INFO:tensorflow:Restoring parameters from log/model/model_60000.ckpt


True

In [9]:
best_acc = 0
saver2 = tf.train.Saver()
train_handle = sess.run(train_iterator.string_handle())
dev_handle = sess.run(dev_iterator.string_handle())

In [10]:
for _ in range(1, 5001):
    loss, train_op = sess.run([model.loss, model.train_op], feed_dict={
                              handle: train_handle})
    
    if _ % 10 == 0:
        print('After', _, 'iterations:')
        print('Batch Loss:', np.mean(loss))
    
    if _ % 100 == 0:
        acc = 0

        for __ in range(18):
            pred1, target1, pred2, target2 = sess.run([tf.argmax(model.y1, 1), model.yp1,
                                                       tf.argmax(model.y2, 1), model.yp2], feed_dict={handle: dev_handle})
            acc += np.mean(np.logical_and(pred1 == target1, pred2 == target2))

        acc /= float(18)
        print('Dev Accuracy:', acc)
        
        if acc > best_acc:
            best_acc = acc
            saver2.save(sess, 'log/retraining_model/badptr-savepoint', global_step=_)
    
    if _ % 10 == 0:
        print()

After 10 iterations:
Batch Loss: 5.023312

After 20 iterations:
Batch Loss: 5.2054114

After 30 iterations:
Batch Loss: 5.9684477

After 40 iterations:
Batch Loss: 6.4695463

After 50 iterations:
Batch Loss: 4.8012667

After 60 iterations:
Batch Loss: 5.213377

After 70 iterations:
Batch Loss: 4.1343207

After 80 iterations:
Batch Loss: 5.0090475

After 90 iterations:
Batch Loss: 4.046558

After 100 iterations:
Batch Loss: 4.827477
Dev Accuracy: 0.265625

After 110 iterations:
Batch Loss: 5.099436

After 120 iterations:
Batch Loss: 4.600661

After 130 iterations:
Batch Loss: 4.1702013

After 140 iterations:
Batch Loss: 4.801069

After 150 iterations:
Batch Loss: 6.073298

After 160 iterations:
Batch Loss: 4.7666554

After 170 iterations:
Batch Loss: 4.774202

After 180 iterations:
Batch Loss: 3.8154802

After 190 iterations:
Batch Loss: 4.8574476

After 200 iterations:
Batch Loss: 5.4550867
Dev Accuracy: 0.2690972222222222

After 210 iterations:
Batch Loss: 5.1139

After 220 iterations

After 1720 iterations:
Batch Loss: 3.0851388

After 1730 iterations:
Batch Loss: 3.225841

After 1740 iterations:
Batch Loss: 3.1558704

After 1750 iterations:
Batch Loss: 2.9225535

After 1760 iterations:
Batch Loss: 3.0339675

After 1770 iterations:
Batch Loss: 3.22522

After 1780 iterations:
Batch Loss: 3.4107833

After 1790 iterations:
Batch Loss: 3.037157

After 1800 iterations:
Batch Loss: 2.5754838
Dev Accuracy: 0.3046875

After 1810 iterations:
Batch Loss: 3.2891583

After 1820 iterations:
Batch Loss: 3.0087492

After 1830 iterations:
Batch Loss: 3.404866

After 1840 iterations:
Batch Loss: 3.6414251

After 1850 iterations:
Batch Loss: 2.9552572

After 1860 iterations:
Batch Loss: 3.4438386

After 1870 iterations:
Batch Loss: 3.129049

After 1880 iterations:
Batch Loss: 3.9131217

After 1890 iterations:
Batch Loss: 3.5301342

After 1900 iterations:
Batch Loss: 3.0042357
Dev Accuracy: 0.3229166666666667

After 1910 iterations:
Batch Loss: 3.4455419

After 1920 iterations:
Batch 

Dev Accuracy: 0.3289930555555556

After 3410 iterations:
Batch Loss: 2.500993

After 3420 iterations:
Batch Loss: 2.7655072

After 3430 iterations:
Batch Loss: 3.0494037

After 3440 iterations:
Batch Loss: 2.7802272

After 3450 iterations:
Batch Loss: 2.951597

After 3460 iterations:
Batch Loss: 2.753364

After 3470 iterations:
Batch Loss: 2.7134147

After 3480 iterations:
Batch Loss: 2.9856763

After 3490 iterations:
Batch Loss: 2.6070507

After 3500 iterations:
Batch Loss: 2.4486864
Dev Accuracy: 0.3289930555555556

After 3510 iterations:
Batch Loss: 2.8249846

After 3520 iterations:
Batch Loss: 2.7808156

After 3530 iterations:
Batch Loss: 2.8523161

After 3540 iterations:
Batch Loss: 2.3943017

After 3550 iterations:
Batch Loss: 2.843918

After 3560 iterations:
Batch Loss: 2.8072677

After 3570 iterations:
Batch Loss: 2.5831926

After 3580 iterations:
Batch Loss: 2.5818696

After 3590 iterations:
Batch Loss: 2.6460965

After 3600 iterations:
Batch Loss: 2.5322602
Dev Accuracy: 0.32